In [2]:
import pandas as pd
import numpy as np

import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [1]:
# Set the path in Google Drive to access the data folder
path = '/home/ukaz/Statapp/data'

### Load data

In [3]:
# Daily market table
markets = pd.read_csv(f'{path}/balance_sheets/daily_markets.csv')
markets['date'] = pd.to_datetime(markets['date'])

In [4]:
markets.columns

Index(['borrowRate', 'cash', 'collateralFactor', 'exchangeRate',
       'interestRateModelAddress', 'name', 'reserves', 'supplyRate', 'symbol',
       'cTokenAddress', 'totalBorrows', 'totalSupply', 'underlyingAddress',
       'underlyingName', 'underlyingPrice', 'underlyingSymbol',
       'accrualBlockNumber', 'blockTimestamp', 'borrowIndex', 'reserveFactor',
       'underlyingPriceUSD', 'underlyingDecimals', 'date', 'newPrice'],
      dtype='object')

In [5]:
markets.head()

,borrowRate,cash,collateralFactor,exchangeRate,interestRateModelAddress,name,reserves,supplyRate,symbol,cTokenAddress,...,underlyingPrice,underlyingSymbol,accrualBlockNumber,blockTimestamp,borrowIndex,reserveFactor,underlyingPriceUSD,underlyingDecimals,date,newPrice
0,0.039471,14.407478,0.50,0.02,0xbae04cbf96391086dc643e842b517734e214d698,Compound Augur,0.000000,0.002306,cREP,0x158079ee67fce2f58472a96584a73c7ab9ac95c1,...,0.116547,REP,7715069,1557250486,1.000041,100000000000000000,20.004143,18,2019-05-07,NaN
1,0.016183,1135.849557,0.75,0.02,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound USD Coin,0.000006,0.001179,cUSDC,0x39aa39c021dfbae8fac545936693ac917d5e7563,...,0.005846,USDC,7716064,1557264374,1.000001,100000000000000000,1.000000,6,2019-05-07,NaN
2,0.012274,15.300218,0.75,0.02,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound Ether,0.000005,0.000678,cETH,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,...,1.000000,ETH,7716242,1557266749,1.000047,100000000000000000,171.043415,18,2019-05-07,NaN
3,0.033222,4381.320658,0.60,0.02,0xbae04cbf96391086dc643e842b517734e214d698,Compound Basic Attention Token,0.000456,0.001318,cBAT,0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e,...,0.001898,BAT,7712572,1557217208,1.000026,100000000000000000,0.335843,18,2019-05-07,NaN
4,0.054585,767.434783,0.60,0.02,0xbae04cbf96391086dc643e842b517734e214d698,Compound 0x,0.000172,0.005664,cZRX,0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407,...,0.001559,ZRX,7711781,1557206685,1.000021,100000000000000000,0.276315,18,2019-05-07,NaN


In [6]:
# Event data
events = list()
for event in ['borrow', 'liquidation', 'mint', 'redeem', 'repay']:
    tmp = pd.read_csv(f'{path}/events/{event}Events_clean.csv')
    tmp['event'] = event

    events.append(tmp)

events = pd.concat(events)
events['blockTime'] = pd.to_datetime(events['blockTime'])
events['date'] = pd.to_datetime(events['blockTime'].dt.date)

In [7]:
# Convert the cTokenAmount/underlyingAmount amount to USD
events = events.merge(markets[['date', 'symbol', 'underlyingPriceUSD', 'exchangeRate']],
             how='left', on=['date', 'symbol'])

events['cTokenAmountUSD'] = events['cTokenAmount'] * events['exchangeRate'] * events['underlyingPriceUSD']
events['underlyingAmountUSD'] = events['underlyingAmount'] * events['underlyingPriceUSD']
events['amountUSD'] = np.where(events['underlyingAmountUSD'].isna(), events['cTokenAmountUSD'], events['underlyingAmountUSD'])

In [8]:
events.columns

Index(['id', 'underlyingAmount', 'accountBorrows', 'account', 'blockNumber',
       'blockTime', 'underlyingSymbol', 'cTokenAddress', 'symbol', 'event',
       'collateralSeizeAmount', 'payer', 'collateralSymbol',
       'collateralCTokenAddress', 'cTokenAmount', 'cTokenSymbol', 'date',
       'underlyingPriceUSD', 'exchangeRate', 'cTokenAmountUSD',
       'underlyingAmountUSD', 'amountUSD'],
      dtype='object')

### How many active users per day?

In [18]:
events.head()

,id,underlyingAmount,accountBorrows,account,blockNumber,blockTime,underlyingSymbol,cTokenAddress,symbol,event,...,collateralSymbol,collateralCTokenAddress,cTokenAmount,cTokenSymbol,date,underlyingPriceUSD,exchangeRate,cTokenAmountUSD,underlyingAmountUSD,amountUSD
0,0x3c0b9a20a119b1a49f1ea105a35fa93e5fca81560ceb...,1.0000,1.0000,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,7711126,2019-05-07 02:51:37,ETH,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,cETH,borrow,...,NaN,NaN,NaN,NaN,2019-05-07,171.043415,0.02,NaN,171.043415,171.043415
1,0x40baa23ee06935a2e7e868f54e4de15a4a0b73b83627...,2.0000,2.0000,0xbd9ed130a53cfafcf81502e4d35329a6c4d53410,7712572,2019-05-07 08:20:08,BAT,0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e,cBAT,borrow,...,NaN,NaN,NaN,NaN,2019-05-07,0.335843,0.02,NaN,0.671687,0.671687
2,0x507ac33d416bfdae472ab83fab83f6c898cbc97b91a0...,200.0000,200.0000,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,7711123,2019-05-07 02:50:47,BAT,0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e,cBAT,borrow,...,NaN,NaN,NaN,NaN,2019-05-07,0.335843,0.02,NaN,67.168667,67.168667
3,0xb1d8d2f1ed5b6d02b2639f4a28528a48f7fd86b697ce...,1.0000,1.0000,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,7715069,2019-05-07 17:34:46,REP,0x158079ee67fce2f58472a96584a73c7ab9ac95c1,cREP,borrow,...,NaN,NaN,NaN,NaN,2019-05-07,20.004143,0.02,NaN,20.004143,20.004143
4,0xbe60c4a7d2add6bf703a09b0b111a1e70014acf1e82c...,0.0003,0.0003,0xa7ff0d561cd15ed525e31bbe0af3fe34ac2059f6,7711672,2019-05-07 04:55:33,ETH,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,cETH,borrow,...,NaN,NaN,NaN,NaN,2019-05-07,171.043415,0.02,NaN,0.051313,0.051313


In [19]:
daily_users = events.groupby(['date', 'event']).agg({
    'account': 'nunique',
    'amountUSD': 'sum'
}).reset_index()

In [20]:
daily_users

,date,event,account,amountUSD
0,2019-05-07,borrow,3,586.964297
1,2019-05-07,mint,10,10143.182859
2,2019-05-08,borrow,3,4171.189123
3,2019-05-08,mint,12,15046.095395
4,2019-05-08,redeem,6,10122.235534
...,...,...,...,...
8065,2023-12-30,repay,6,0.000000
8066,2023-12-31,borrow,20,0.000000
8067,2023-12-31,mint,21,0.000000
8068,2023-12-31,redeem,34,0.000000


In [25]:
graph = alt.Chart(daily_users).mark_area().encode(
    x='date:T',
    y='account:Q',
    color='event',
    tooltip=['event', 'account:Q', 'amountUSD:Q']
).interactive(bind_y = False).save('graphity.png')



In [28]:
# Compare borrow and repay amount
alt.Chart(daily_users).mark_line().encode(
    x='date:T',
    y=alt.Y('amountUSD:Q', axis=alt.Axis(format='s')),
    color='event:N',
    tooltip=['event:N', 'account:Q', 'amountUSD:Q']
).interactive(bind_y = False).save('second_try.html')

In [ ]:
# You can also further divided by the symbol
events.groupby(['date', 'event', 'symbol']).agg({
    'account': 'nunique',
    'amountUSD': 'sum'
}).reset_index()

,date,event,symbol,account,amountUSD
0,2019-05-07,borrow,cBAT,2,67.840354
1,2019-05-07,borrow,cETH,2,171.094728
2,2019-05-07,borrow,cREP,1,20.004143
3,2019-05-07,borrow,cSAI,1,200.393556
4,2019-05-07,borrow,cUSDC,1,100.000000
...,...,...,...,...,...
57693,2023-12-22,redeem,cWBTC2,2,0.000000
57694,2023-12-22,repay,cDAI,2,0.000000
57695,2023-12-22,repay,cETH,6,0.000000
57696,2023-12-22,repay,cUSDC,6,0.000000
